<a href="https://colab.research.google.com/github/alexk2206/Data_Driven_Fantasy_Football/blob/dev/Enhanced_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [95]:
!pip install mip
import pandas as pd
import numpy as np
import re
import random
from mip import Model, BINARY, CONTINUOUS, xsum, maximize

# Create Player dataset

In [106]:
file_path = 'https://raw.githubusercontent.com/alexk2206/Data_Driven_Fantasy_Football/refs/heads/dev/FantasyPros_2024_Overall_ADP_Rankings.csv'
df = pd.read_csv(file_path)#, on_bad_lines='skip')
df['POS'] = df['POS'].str.replace('\d+', '', regex=True)

def extract_numbers(s):
    if pd.isna(s):
        return None
    numbers = re.findall(r'\d+', str(s))
    if numbers:
        return int(numbers[0])
    return None

num_of_players = len(df) #250

projections_df = df[['Player', 'Bye', 'POS', 'AVG']].head(num_of_players).copy()
projections_df['Bye'] = projections_df['Bye'].apply(extract_numbers)
projections_df['Bye'] = projections_df['Bye'].fillna(0).astype(int)

projections_df.info()
print(projections_df.value_counts('POS'))
print(projections_df.head(20))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 948 entries, 0 to 947
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Player  948 non-null    object 
 1   Bye     948 non-null    int64  
 2   POS     948 non-null    object 
 3   AVG     948 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 29.8+ KB
POS
WR     335
RB     224
TE     166
QB     127
K       64
DST     32
Name: count, dtype: int64
                 Player  Bye POS   AVG
0   Christian McCaffrey    9  RB   1.0
1           CeeDee Lamb    7  WR   2.6
2           Tyreek Hill    6  WR   3.2
3        Bijan Robinson   12  RB   5.0
4           Breece Hall   12  RB   5.4
5     Amon-Ra St. Brown    5  WR   6.2
6         Ja'Marr Chase   12  WR   6.6
7      Justin Jefferson    6  WR   7.0
8        Saquon Barkley    5  RB   9.2
9            A.J. Brown    5  WR  10.2
10      Jonathan Taylor   14  RB  10.4
11       Garrett Wilson   12  WR  12.4
12      

# Create custom projections
Maybe delete later when real projections available

In [107]:
# Anzahl der Wochen
number_of_weeks = 17

# Wochen-Spaltennamen
weekly_columns = [f"Week_{i+1}" for i in range(number_of_weeks)]

# Skalierungsfunktion
def final_projection_base(avg, pos, max_val=22, min_val=7, k=50, c=1.5):
    base = min_val + (max_val - min_val) * (1 / (1 + (avg / k) ** c))
    if pos == "QB":
        base += 4  # QB-Bonus
    elif pos == "K":
        base -= 3  # K-Penalty
    elif pos == "DST":
        base -= 3  # DST-Penalty
    return base

final_projections = []
for _, row in projections_df.iterrows():
    base_score = final_projection_base(row['AVG'], row['POS'])

    # Erstellen der weekly projections
    weekly_proj = []
    for week in range(number_of_weeks):
        # Überprüfen, ob die aktuelle Woche (week + 1) mit der Bye-Woche des Spielers übereinstimmt
        if (week + 1) == row['Bye']:  # Woche des Spielers = Bye-Woche?
            weekly_proj.append(0.0)  # Projektion auf 0 setzen
        else:
            weekly_proj.append(base_score + np.random.normal(0, base_score * 0.1))  # Zufällige Variation

    final_projections.append(weekly_proj)

# Projektionen in DataFrame einfügen
f = projections_df[["Player", "Bye", "POS", "AVG"]].copy()
for i, col in enumerate(weekly_columns):
    f[col] = [proj[i] for proj in final_projections]

# Zeige eine zufällige Stichprobe der ersten 25 Zeilen
f.head(25)

,Player,Bye,POS,AVG,Week_1,Week_2,Week_3,Week_4,Week_5,Week_6,...,Week_8,Week_9,Week_10,Week_11,Week_12,Week_13,Week_14,Week_15,Week_16,Week_17
0,Christian McCaffrey,9,RB,1.0,26.027980,20.892139,24.158089,22.569644,23.407500,24.287035,...,21.491098,0.000000,18.100165,22.168344,19.217394,22.259211,24.613042,24.374422,20.077076,20.803625
1,CeeDee Lamb,7,WR,2.6,21.109895,23.435275,19.496758,20.488536,24.064189,23.050296,...,21.343932,21.816496,22.142189,23.439943,23.784932,22.922132,22.317265,25.742206,23.901728,22.458232
2,Tyreek Hill,6,WR,3.2,19.199037,20.113189,25.127718,26.029816,21.283997,0.000000,...,21.481475,21.437574,19.585705,19.557704,24.531751,18.176022,25.064755,22.040504,23.059226,20.382913
3,Bijan Robinson,12,RB,5.0,17.885010,18.939544,22.322262,22.436682,24.162188,22.274631,...,22.374598,23.850667,20.095402,22.142005,0.000000,21.909248,21.919586,23.007353,25.472208,24.158632
4,Breece Hall,12,RB,5.4,18.305962,22.679722,22.583817,21.551137,16.877651,22.955923,...,21.234519,21.179192,18.720849,21.880201,0.000000,21.392403,20.205020,24.544742,24.765562,15.554339
5,Amon-Ra St. Brown,5,WR,6.2,24.099265,21.304365,25.773187,21.195550,0.000000,23.271338,...,23.094511,20.031132,22.364648,22.856501,19.931371,21.677122,19.696993,21.361344,19.384577,22.354014
6,Ja'Marr Chase,12,WR,6.6,21.796207,20.138739,23.479381,19.609860,18.976985,19.268299,...,21.559092,20.906972,23.048838,20.726104,0.000000,25.625666,21.911082,21.511937,27.812189,24.290491
7,Justin Jefferson,6,WR,7.0,22.274480,22.134507,22.561035,20.081064,24.545850,0.000000,...,18.467416,20.672120,23.019002,22.166429,20.276640,21.985553,18.199210,18.700975,24.080522,22.083443
8,Saquon Barkley,5,RB,9.2,23.048893,21.339995,21.861689,18.517099,0.000000,22.428543,...,21.886357,23.873535,24.232549,16.848644,20.912860,20.799181,17.720223,18.110436,20.427304,20.302764
9,A.J. Brown,5,WR,10.2,22.505384,20.669263,21.212300,18.232467,0.000000,19.057006,...,23.132738,22.396339,18.860120,23.623898,17.188042,14.750651,21.009543,20.915198,19.063290,22.306794


In [108]:
# --- 1. Parameter (aus deinem paste-2.txt) ---
players        = projections_df['Player'].tolist()
positions      = {"QB", "RB", "WR", "TE", "K", "DST"}
weeks          = list(range(1,18))
pos            = dict(zip(projections_df['Player'], projections_df['POS']))
pos_limit      = {"QB":1,"RB":2,"WR":2,"TE":1,"K":1,"DST":1}
week_cols = [col for col in f.columns if col.startswith("Week_")]
f_dict = {
    row['Player']: {int(week.replace("Week_", "")): row[week] for week in week_cols}
    for _, row in f.iterrows()
}
beta           = {t:100.0 for t in weeks}
gamma          = {"QB":2,"RB":3,"WR":3,"TE":2,"K":1,"DST":1}
alpha, lambda_0, lambda_1, lambda_2 = 1.0, 1, 100, 150
df_sorted = projections_df.sort_values("AVG").reset_index(drop=True)
df_sorted["Rank"] = df_sorted.index + 1
# Mindest‐Anzahl Spieler pro Position im Roster
min_pos_req = pos_limit


# --- 2. Teams, DM-Team und Snake-Draft ---
num_teams      = 12
teams          = [f"Team {i+1}" for i in range(num_teams)]
DM_team        = "Team 1"
num_rounds     = 15
draft_order    = []
for rnd in range(num_rounds):
    order = teams if rnd % 2 == 0 else teams[::-1]
    draft_order += order

def opponent_pick(roster, available, Rk, min_pos_req, topk_pct=0.01):
    # 1) verbleibende Spieler neu sortieren
    rem  = sorted(available, key=lambda p: Rk[p])
    topk = max(1, int(len(rem) * topk_pct))

    # 2) Defizite je Position (Mindestsoll minus aktueller Bestand)
    deficits = {
        j: min_pos_req[j] - sum(1 for p in roster if pos[p] == j)
        for j in min_pos_req
    }
    needed = [j for j, d in deficits.items() if d > 0]

    # 3) solange Defizite bestehen, aus allen rem dieser Position picken
    if needed:
        # Kandidaten aller benötigten Positionen
        candidates = [p for p in rem if pos[p] in needed]
        # begrenze auf Top-k, falls mehr Kandidaten vorhanden
        pool = candidates[:topk] if len(candidates) >= topk else candidates
        if pool:
            return random.choice(pool)

    # 4) Fallback: zufällig aus Top-k aller Positionen
    return random.choice(rem[:topk])

# --- 3. Initialisierung ---
rosters        = {tm: [] for tm in teams}
available      = set(players)
draft_log      = []

# --- 4. Hauptschleife über alle Picks ---
for pick_idx, team in enumerate(draft_order, start=1):
    # 4.1 Ranking der verbleibenden Spieler aktualisieren
    rem = sorted(available, key=lambda p: df_sorted.loc[df_sorted.Player==p,"AVG"].item())
    Rk  = {p: i+1 for i,p in enumerate(rem)}
    player_vars = set(rem) | set(rosters[team])


    # 4.2 DM-Pick via MIP
    if team == DM_team:
        m = Model(sense=maximize, solver_name="CBC")

        # Variablen nur für verbleibende Spieler
        y = {i: m.add_var(var_type=BINARY, name=f"y_{i}")
              for i in player_vars}
        x = {(i,t): m.add_var(var_type=CONTINUOUS, name=f"x_{i}_{t}")
             for i in rem for t in weeks}
        z = {t: m.add_var(var_type=BINARY, name=f"z_{t}") for t in weeks}

        # Fixiere vergangene Picks
        for p in rosters[team]:
            m += y[p] == 1

        # Objective
        m.objective = (
            lambda_0 * xsum(f_dict[i][t]*x[i,t] for i in rem for t in weeks)
          + lambda_1 * xsum(z[t] for t in weeks[:15])
          + lambda_2 * xsum(z[t] for t in weeks[15:])
        )

        # Position‐ und Roster‐Constraints
        for j in positions:
            # Müssen mindestens gamma[j] ziehen
            m += xsum(y[i] for i in rem if pos[i]==j) >= gamma[j]
            for t in weeks:
                m += xsum(x[i,t] for i in rem if pos[i]==j) <= pos_limit[j]
        # Roster Anforderungen
        for pos_name, req in min_pos_req.items():
            m.add_constr(
                xsum(y[i] for i in player_vars if pos.get(i) == pos_name) >= req,
                name=f"min_roster_{pos_name}")
        # maximale Picks pro Team
        m += xsum(y[i] for i in rem) <= num_rounds
        # nur gedraftete Spieler in der Week‐Lineup
        for i in rem:
            for t in weeks:
                m += x[i,t] <= y[i]
        # Win‐Indicator
        for t in weeks:
            m += z[t] <= xsum(f_dict[i][t]*x[i,t] for i in rem) / beta[t]

        # Robuste Draft‐Constraint (1b) – grob implementiert
        n_k = pick_idx
        for future_pick in range(pick_idx+1, pick_idx + (num_rounds - len(rosters[team]))*len(teams), len(teams)):
            top_cut = int(alpha*(future_pick - n_k))
            if top_cut > 0:
                top_players = [i for i,r in Rk.items() if r <= top_cut]
                m += xsum(y[i] for i in top_players) <= ( (future_pick-n_k) // len(teams) )

        m.optimize()
        # gewählten Spieler extrahieren
        chosen = [i for i in rem if y[i].x >= 0.99 and i not in rosters[team]]
        if not chosen:
            raise RuntimeError(f"No feasible pick at {pick_idx}")
        pick = min(chosen, key=lambda i: Rk[i])

    # 4.3 Gegner-Pick: zufällig aus Top-5 verbleibend
    else:
        pick = opponent_pick(roster=rosters[team], available=available, Rk=Rk, min_pos_req=pos_limit, topk_pct=0.01)

    # 4.4 Update
    rosters[team].append(pick)
    available.remove(pick)
    draft_log.append({
        "Pick": pick_idx, "Team": team, "Player": pick,
        "Round": (pick_idx-1)//len(teams)+1, "POS": pos[pick]
    })

# --- 5. Ergebnis als DataFrame ---
df_draft = pd.DataFrame(draft_log)
print(df_draft.head(1+num_teams*2))


    Pick     Team               Player  Round POS
0      1   Team 1         Stefon Diggs      1  WR
1      2   Team 2  Christian McCaffrey      1  RB
2      3   Team 3       Saquon Barkley      1  RB
3      4   Team 4    Amon-Ra St. Brown      1  WR
4      5   Team 5          CeeDee Lamb      1  WR
5      6   Team 6       Garrett Wilson      1  WR
6      7   Team 7           A.J. Brown      1  WR
7      8   Team 8         Jahmyr Gibbs      1  RB
8      9   Team 9          Tyreek Hill      1  WR
9     10  Team 10       Bijan Robinson      1  RB
10    11  Team 11        Ja'Marr Chase      1  WR
11    12  Team 12        Isiah Pacheco      1  RB
12    13  Team 12          Breece Hall      2  RB
13    14  Team 11   Travis Etienne Jr.      2  RB
14    15  Team 10         Travis Kelce      2  TE
15    16   Team 9        Derrick Henry      2  RB
16    17   Team 8        Davante Adams      2  WR
17    18   Team 7  Marvin Harrison Jr.      2  WR
18    19   Team 6          Chris Olave      2  WR


In [105]:
print(m.status)
print(m.num_constrs, m.num_vars)
print(m)

OptimizationStatus.OPTIMAL


AttributeError: 'Model' object has no attribute 'num_constrs'

In [109]:
position_counts_df = df_draft.groupby(['Team', 'POS']).size().unstack(fill_value=0)
position_counts_df

POS,DST,K,QB,RB,TE,WR
Team,,,,,,
Team 1,1,2,5,2,0,5
Team 10,1,1,1,5,2,5
Team 11,1,1,1,5,2,5
Team 12,1,1,2,3,1,7
Team 2,1,1,3,4,2,4
Team 3,1,1,2,4,1,6
Team 4,2,1,2,5,2,3
Team 5,1,1,1,3,3,6
Team 6,1,1,1,4,2,6


In [115]:
result_dfs = {}

for team in df_draft['Team'].unique():
    team_df = df_draft[df_draft['Team'] == team].copy()
    team_df = team_df.sort_values(by='Pick')

    # Optional: Spalten anpassen, wenn nur bestimmte Infos gewünscht sind
    team_df['Pick Info'] = team_df.apply(lambda row: f"Round {row['Round']} Pick {row['Pick']}", axis=1)
    result_dfs[team] = team_df[['Player', 'Pick Info', 'POS']]  # oder andere gewünschte Spalten

for team, df in result_dfs.items():
    print(f"=== {team} ===")
    print(df)
    print()


=== Team 1 ===
              Player          Pick Info  POS
0       Stefon Diggs     Round 1 Pick 1   WR
23     Xavier Worthy    Round 2 Pick 24   WR
24    Tua Tagovailoa    Round 3 Pick 25   QB
47    Christian Kirk    Round 4 Pick 48   WR
48   Diontae Johnson    Round 5 Pick 49   WR
71     Chuba Hubbard    Round 6 Pick 72   RB
72         Zack Moss    Round 7 Pick 73   RB
95    Dustin Hopkins    Round 8 Pick 96    K
96   Trevor Lawrence    Round 9 Pick 97   QB
119    New York Jets  Round 10 Pick 120  DST
120    Aaron Rodgers  Round 11 Pick 121   QB
143     Kirk Cousins  Round 12 Pick 144   QB
144    Khalil Shakir  Round 13 Pick 145   WR
167   Deshaun Watson  Round 14 Pick 168   QB
168     Cairo Santos  Round 15 Pick 169    K

=== Team 2 ===
                     Player          Pick Info  POS
1       Christian McCaffrey     Round 1 Pick 2   RB
22          Jonathan Taylor    Round 2 Pick 23   RB
25              Sam LaPorta    Round 3 Pick 26   TE
46            Jaylen Waddle    Round 4 Pi